# Import Packages

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split

import pandas as pd, numpy as np
from patsy import dmatrices, dmatrix, demo_data

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import snowball
from nltk.tag import pos_tag
from nltk.tokenize import WordPunctTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import PCA

from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist

import os
from scipy.cluster.hierarchy import fcluster

import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.neighbors import KernelDensity

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from nltk.corpus import stopwords

import pandas as pd, seaborn as sns, numpy as np, matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB

from sqlalchemy import create_engine

from sklearn.linear_model import ElasticNet
#for retina display only
#%config InlineBackend.figure_format = 'retina'

from datetime import timedelta

import prettyplotlib as ppl


# Load Data

In [2]:
spray = pd.read_csv('./data/spray.csv')
weather = pd.read_csv('./data/weather.csv')
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')

__Link:__
https://www.kaggle.com/c/predict-west-nile-virus/data

__TABLE:__ spray.csv - GIS data of spraying efforts in 2011 and 2013

##### SPRAY PARAMETERS Description:
__Date, Time:__ the date and time of the spray
__Latitude, Longitude:__ the Latitude and Longitude of the spray

In [3]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


__TABLES:__ train.csv, test.csv - the training and test set of the main dataset. 
train.csv, test.csv - the training and test set of the main dataset. 
The training set consists of data from 2007, 2009, 2011, and 2013, 
while in the test set you are requested to predict the test results for 2008, 2010, 2012, and 2014.

##### TRAIN TEST PARAMETERS Description:

__Id:__ the id of the record
__Date:__ date that the WNV test is performed
__Address:__ approximate address of the location of trap. This is used to send to the GeoCoder. 
__Species:__ the species of mosquitos
__Block:__ block number of address
__Street:__ street name
__Trap:__ Id of the trap
__AddressNumberAndStreet:__ approximate address returned from GeoCoder
__Latitude, Longitude:__ Latitude and Longitude returned from GeoCoder
__AddressAccuracy:__ accuracy returned from GeoCoder
__NumMosquitos:__ number of mosquitoes caught in this trap
__WnvPresent:__ whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 

In [4]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [5]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


__TABLES:___ weather.csv - weather data from 2007 to 2014. 
Column descriptions in noaa_weather_qclcd_documentation.pdf. 

In [6]:
station_1_weather = weather[weather['Station']==1]

In [7]:
merged_s1_train = pd.merge(station_1_weather,train,how='right', on='Date')

In [8]:
merged_s1_train.head()


,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,1,2007-05-29,88,60,74,10,58,65,0,9,...,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,1,2007-05-29,88,60,74,10,58,65,0,9,...,CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,1,2007-05-29,88,60,74,10,58,65,0,9,...,CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,1,2007-05-29,88,60,74,10,58,65,0,9,...,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,1,2007-05-29,88,60,74,10,58,65,0,9,...,CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [9]:
merged_s1_train.WetBulb = merged_s1_train.WetBulb.map(lambda x: np.NaN if x=='M' else x)
merged_s1_train.dropna(inplace=True)
merged_s1_train.WetBulb.value_counts()
merged_s1_train.WetBulb = merged_s1_train.WetBulb.map(lambda x: int(x))

In [10]:
merged_s1_train.Heat.value_counts()
merged_s1_train.Heat = merged_s1_train.Heat.map(lambda x: int(x))

In [11]:
merged_s1_train.dtypes

Station                     int64
Date                       object
Tmax                        int64
Tmin                        int64
Tavg                       object
Depart                     object
DewPoint                    int64
WetBulb                     int64
Heat                        int64
Cool                       object
Sunrise                    object
Sunset                     object
CodeSum                    object
Depth                      object
Water1                     object
SnowFall                   object
PrecipTotal                object
StnPressure                object
SeaLevel                   object
ResultSpeed               float64
ResultDir                   int64
AvgSpeed                   object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude     

In [12]:
merged_s1_train['Tavg'] = merged_s1_train['Tavg'].map(lambda x: float(x))
merged_s1_train['Depart'] = merged_s1_train['Depart'].map(lambda x: float(x))
merged_s1_train['AvgSpeed'] = merged_s1_train['AvgSpeed'].map(lambda x: float(x))
merged_s1_train['Heat'] = merged_s1_train['Heat'].map(lambda x: float(x))
merged_s1_train['Cool'] = merged_s1_train['Cool'].map(lambda x: float(x))
merged_s1_train['Depth'] = merged_s1_train['Depth'].map(lambda x: float(x))
merged_s1_train['ResultDir'] = merged_s1_train['ResultDir'].map(lambda x: float(x))
merged_s1_train['StnPressure'] = merged_s1_train['StnPressure'].map(lambda x: float(x))
merged_s1_train['SeaLevel'] = merged_s1_train['SeaLevel'].map(lambda x: float(x))
#merged_s1_train['Water1'] = merged_s1_train['Water1'].map(lambda x: float(x))
#merged_s1_train['PrecipTotal'] = merged_s1_train['Water1'].map(lambda x: float(x)) #take care of it below

In [13]:
merged_s1_train.Water1.value_counts() #Water1, object, will drop it
merged_s1_train.drop('Water1', axis=1, inplace=True)

In [14]:
merged_s1_train.SnowFall.value_counts() #SnowFall, object, will drop it
merged_s1_train.drop('SnowFall', axis=1, inplace=True)

In [15]:
#Convert Date time
merged_s1_train.Date = pd.to_datetime(merged_s1_train.Date)

In [16]:
merged_s1_train.dtypes

Station                            int64
Date                      datetime64[ns]
Tmax                               int64
Tmin                               int64
Tavg                             float64
Depart                           float64
DewPoint                           int64
WetBulb                            int64
Heat                             float64
Cool                             float64
Sunrise                           object
Sunset                            object
CodeSum                           object
Depth                            float64
PrecipTotal                       object
StnPressure                      float64
SeaLevel                         float64
ResultSpeed                      float64
ResultDir                        float64
AvgSpeed                         float64
Address                           object
Species                           object
Block                              int64
Street                            object
Trap            

In [17]:
merged_s1_train.SeaLevel.value_counts()

30.00    856
29.98    741
29.91    642
30.05    525
29.94    494
29.83    475
29.89    445
29.95    421
29.82    415
30.04    393
30.11    361
30.10    341
29.76    334
30.06    295
29.93    292
29.99    287
29.87    276
30.12    260
30.01    236
29.97    230
30.09    224
30.13    203
29.86    195
30.07    182
29.84    155
29.92    127
30.18    127
30.33    120
29.79    112
29.90    110
29.70     93
29.96     91
30.22     74
29.73     62
29.63     60
30.16     55
30.08     54
29.60     31
30.15     14
30.02      5
Name: SeaLevel, dtype: int64

In [18]:
merged_s1_train.StnPressure.value_counts()

29.34    811
29.29    786
29.28    735
29.23    639
29.36    516
29.21    509
29.38    430
29.26    424
29.11    398
29.18    384
29.13    374
29.33    346
29.27    344
29.20    318
29.24    303
29.40    292
29.17    276
29.06    243
29.10    242
29.14    224
29.41    223
29.39    220
29.08    184
29.15    164
29.12    146
29.22    128
29.45    127
29.59    120
28.97     93
29.49     74
29.42     69
29.01     62
29.32     61
28.91     60
29.09     52
28.89     31
29.31      5
Name: StnPressure, dtype: int64

In [19]:
#PrecipTotal, '  T' "Trace Amounts" = 0.0
merged_s1_train.PrecipTotal = merged_s1_train.PrecipTotal.map(lambda x: 0.00 if x=='  T' else x)
merged_s1_train.PrecipTotal = merged_s1_train.PrecipTotal.map(lambda x: float(x))
merged_s1_train.PrecipTotal.value_counts()

0.00    7002
0.23     443
0.16     240
0.24     186
1.31     184
0.58     182
0.39     146
0.37     139
0.70     138
0.33     131
1.55     129
0.50     119
0.79     116
0.09     113
0.84     112
0.08      93
3.97      93
0.12      93
0.14      93
0.06      93
0.02      83
0.17      83
0.89      65
0.52      62
0.92      62
0.42      60
0.95      54
0.18      50
0.05      26
0.27      23
Name: PrecipTotal, dtype: int64

In [20]:
merged_s1_train.dtypes

Station                            int64
Date                      datetime64[ns]
Tmax                               int64
Tmin                               int64
Tavg                             float64
Depart                           float64
DewPoint                           int64
WetBulb                            int64
Heat                             float64
Cool                             float64
Sunrise                           object
Sunset                            object
CodeSum                           object
Depth                            float64
PrecipTotal                      float64
StnPressure                      float64
SeaLevel                         float64
ResultSpeed                      float64
ResultDir                        float64
AvgSpeed                         float64
Address                           object
Species                           object
Block                              int64
Street                            object
Trap            

In [21]:
merged_s1_train.head(3).T

,0,1,2
Station,1,1,1
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-05-29 00:00:00
Tmax,88,88,88
Tmin,60,60,60
Tavg,74,74,74
Depart,10,10,10
DewPoint,58,58,58
WetBulb,65,65,65
Heat,0,0,0
Cool,9,9,9


In [22]:
merged_s1_train.columns

Index([u'Station', u'Date', u'Tmax', u'Tmin', u'Tavg', u'Depart', u'DewPoint',
       u'WetBulb', u'Heat', u'Cool', u'Sunrise', u'Sunset', u'CodeSum',
       u'Depth', u'PrecipTotal', u'StnPressure', u'SeaLevel', u'ResultSpeed',
       u'ResultDir', u'AvgSpeed', u'Address', u'Species', u'Block', u'Street',
       u'Trap', u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy', u'NumMosquitos', u'WnvPresent'],
      dtype='object')

In [23]:
#merged_s1_train.drop('Station',axis=1, inplace=True)
merged_s1_train.drop('Depth',axis=1, inplace=True)

In [36]:
merged_s1_train.to_csv("./merged_s1_train.csv", encoding='utf-8')

In [24]:
merged_s1_train.dtypes

Station                            int64
Date                      datetime64[ns]
Tmax                               int64
Tmin                               int64
Tavg                             float64
Depart                           float64
DewPoint                           int64
WetBulb                            int64
Heat                             float64
Cool                             float64
Sunrise                           object
Sunset                            object
CodeSum                           object
PrecipTotal                      float64
StnPressure                      float64
SeaLevel                         float64
ResultSpeed                      float64
ResultDir                        float64
AvgSpeed                         float64
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAnd

In [25]:
import patsy
from patsy import dmatrices

formula = 'WnvPresent ~ Tmax + Tmin + Tavg + Depart + DewPoint + WetBulb + Heat + Cool + C(Sunrise) + C(Sunset) + C(CodeSum)\
           + PrecipTotal + StnPressure + SeaLevel + ResultSpeed + ResultDir + AvgSpeed + C(Address) + C(Species) + Block\
           + C(Street) + C(Trap) + C(AddressNumberAndStreet) + Latitude + Longitude + AddressAccuracy + NumMosquitos -1'

formula = str(formula)

Yb, Xb = dmatrices(formula, data = merged_s1_train, return_type = 'dataframe')

In [26]:
Xb_mat = (Xb - Xb.mean())/Xb.std()

In [27]:
from sklearn.metrics import accuracy_score


logreg = LogisticRegression()

model = logreg.fit(Xb_mat, Yb)

predictions = logreg.predict(Xb_mat)

print accuracy_score(Yb, predictions)

/Users/jrcmalunay/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.948141745895


In [35]:
t = logreg.coef_
tt = t.reshape(-1)

house_coefs = pd.DataFrame({'feature' : Xb.columns, 'coef':tt})

house_coefs.sort(columns='coef',ascending=False)

/Users/jrcmalunay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,coef,feature
687,0.870155,PrecipTotal
697,0.751977,NumMosquitos
690,0.565000,ResultSpeed
123,0.470101,C(CodeSum)[T.RA]
685,0.439685,Heat
679,0.410424,Tmax
686,0.318251,Cool
92,0.312662,C(Sunset)[T.1854]
30,0.312662,C(Sunrise)[0459]
21,0.258129,C(Sunrise)[0445]


In [29]:
from sklearn.linear_model import ElasticNetCV

In [30]:
import numpy as np
enet = ElasticNetCV()

params = {
    'l1_ratio': np.linspace(0,1,100)
}

grid_EN = GridSearchCV(enet,params,cv=5)

In [31]:
Y = merged_s1_train.WnvPresent
X = merged_s1_train[[x for x in merged_s1_train.columns if x in ['Tmin','Tmax','DewPoint','ResultSpeed','ResultDir',]]]

grid_EN.fit(X,Y)


UnboundLocalError: local variable 'best_l1_ratio' referenced before assignment

In [ ]:
station_2_weather = weather[weather['Station']==2]
station_2_weather

In [ ]:
weather.Sunset.unique()

In [ ]:
weather.Sunrise.unique()

In [ ]:
spray = spray.dropna()

In [ ]:
spray.dtypes

In [ ]:
#weather.CodeSum = [s.replace(" ","MODERATE") for s in weather.CodeSum]
weather.CodeSum = weather.CodeSum.map(lambda x: "MODERATE" if x == " " else x)
#weather.CodeSum.value_counts()

In [ ]:
train.dtypes

In [ ]:
test.dtypes